# 実践演習 Day 1：streamlitとFastAPIのデモ
このノートブックでは以下の内容を学習します。

- 必要なライブラリのインストールと環境設定
- Hugging Faceからモデルを用いたStreamlitのデモアプリ
- FastAPIとngrokを使用したAPIの公開方法

演習を始める前に、HuggingFaceとngrokのアカウントを作成し、
それぞれのAPIトークンを取得する必要があります。


演習の時間では、以下の3つのディレクトリを順に説明します。

1. 01_streamlit_UI
2. 02_streamlit_app
3. 03_FastAPI

2つ目や3つ目からでも始められる様にノートブックを作成しています。

復習の際にもこのノートブックを役立てていただければと思います。

### 注意事項
「02_streamlit_app」と「03_FastAPI」では、GPUを使用します。

これらを実行する際は、Google Colab画面上のメニューから「編集」→ 「ノートブックの設定」

「ハードウェアアクセラレーター」の項目の中から、「T4 GPU」を選択してください。

このノートブックのデフォルトは「CPU」になっています。

---

# 環境変数の設定（1~3共有）


GitHubから演習用のコードをCloneします。

In [ ]:
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

必要なAPIトークンを.envに設定します。

「lecture-ai-engineering/day1」の配下に、「.env_template」ファイルが存在しています。

隠しファイルのため表示されていない場合は、画面左側のある、目のアイコンの「隠しファイルの表示」ボタンを押してください。

「.env_template」のファイル名を「.env」に変更します。「.env」ファイルを開くと、以下のような中身になっています。


```
HUGGINGFACE_TOKEN="hf-********"
NGROK_TOKEN="********"
```
ダブルクオーテーションで囲まれた文字列をHuggingfaceのアクセストークンと、ngrokの認証トークンで書き変えてください。

それぞれのアカウントが作成済みであれば、以下のURLからそれぞれのトークンを取得できます。

- Huggingfaceのアクセストークン
https://huggingface.co/docs/hub/security-tokens

- ngrokの認証トークン
https://dashboard.ngrok.com/get-started/your-authtoken

書き換えたら、「.env」ファイルをローカルのPCにダウンロードしてください。

「01_streamlit_UI」から「02_streamlit_app」へ進む際に、CPUからGPUの利用に切り替えるため、セッションが一度切れてしまいます。

その際に、トークンを設定した「.env」ファイルは再作成することになるので、その手間を減らすために「.env」ファイルをダウンロードしておくと良いです。

「.env」ファイルを読み込み、環境変数として設定します。次のセルを実行し、最終的に「True」が表示されていればうまく読み込めています。

In [1]:
!pip install python-dotenv
from dotenv import load_dotenv, find_dotenv


load_dotenv(find_dotenv())

True

# 01_streamlit_UI

ディレクトリ「01_streamlit_UI」に移動します。

In [2]:
!pwd
%cd 01_streamlit_UI/
# !cd
!pwd

/home/tmiya/src/github.com/silbull/lecture-ai-engineering/day1
/home/tmiya/src/github.com/silbull/lecture-ai-engineering/day1/01_streamlit_UI
/home/tmiya/src/github.com/silbull/lecture-ai-engineering/day1/01_streamlit_UI


必要なライブラリをインストールします。

In [3]:
%%capture
!pip install -r requirements.txt

ngrokのトークンを使用して、認証を行います。

In [4]:
!ngrok authtoken $$NGROK_TOKEN

Authtoken saved to configuration file: /home/tmiya/.config/ngrok/ngrok.yml


In [6]:
!pip list

Package                   Version
------------------------- -----------
altair                    5.5.0
asttokens                 3.0.0
attrs                     25.3.0
blinker                   1.9.0
cachetools                5.5.2
certifi                   2025.4.26
charset-normalizer        3.4.1
click                     8.1.8
comm                      0.2.2
debugpy                   1.8.14
decorator                 5.2.1
executing                 2.2.0
gitdb                     4.0.12
GitPython                 3.1.44
idna                      3.10
ipykernel                 6.29.5
ipython                   9.2.0
ipython_pygments_lexers   1.1.1
jedi                      0.19.2
Jinja2                    3.1.6
jsonschema                4.23.0
jsonschema-specifications 2025.4.1
jupyter_client            8.6.3
jupyter_core              5.7.2
MarkupSafe                3.0.2
matplotlib-inline         0.1.7
narwhals                  1.37.1
nest-asyncio              1.6.0
numpy             

アプリを起動します。(ローカル用)
colabでは即時にポートが開くようになっているが、ローカルではポートを開くのに数秒かかるためこのような処理にしている。


In [14]:
import subprocess
import time
from pyngrok import ngrok

# streamlit起動（8501番ポート）
proc = subprocess.Popen(["streamlit", "run", "app.py"])

# 起動をちょっと待つ（本来はポート確認がベター）
time.sleep(5)

# ngrokトンネル作成
public_url = ngrok.connect(8501)
print(f"🔗 公開URL: {public_url}")

try:
    proc.wait()
except KeyboardInterrupt:
    print("🛑 終了中...")
    proc.terminate()
    ngrok.kill()
    print("🔒 ngrokトンネルを終了しました。")


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.22.108.179:8501



gio: http://localhost:8501: Operation not supported


🔗 公開URL: NgrokTunnel: "https://b474-61-114-134-7.ngrok-free.app" -> "http://localhost:8501"


t=2025-04-30T00:51:42+0900 lvl=warn msg="Stopping forwarder" name=http-8501-7d122a20-bf0e-41a1-9d82-55950f458cb1 acceptErr="failed to accept connection: Listener closed"
t=2025-04-30T00:51:42+0900 lvl=warn msg="Error restarting forwarder" name=http-8501-7d122a20-bf0e-41a1-9d82-55950f458cb1 err="failed to start tunnel: session closed"


  Stopping...
🛑 終了中...
🔒 ngrokトンネルを終了しました。


Colab用(デフォルト)

In [10]:
from pyngrok import ngrok

public_url = ngrok.connect(8500).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

公開URL: https://8b05-61-114-134-7.ngrok-free.app

      👋 Welcome to Streamlit!

      If you’d like to receive helpful onboarding emails, news, offers, promotions,
      and the occasional swag, please enter your email address below. Otherwise,
      leave this field blank.

      Email:  

t=2025-04-30T00:39:58+0900 lvl=warn msg="failed to open private leg" id=e570026526b0 privaddr=localhost:8500 err="dial tcp 127.0.0.1:8500: connect: connection refused"
t=2025-04-30T00:39:58+0900 lvl=warn msg="failed to open private leg" id=37bb40968449 privaddr=localhost:8500 err="dial tcp 127.0.0.1:8500: connect: connection refused"
t=2025-04-30T00:40:47+0900 lvl=warn msg="Stopping forwarder" name=http-8500-792da211-eb73-4927-8e7b-f7f2a289f1c3 acceptErr="failed to accept connection: Listener closed"


2025-04-30 00:40:47.435 
^C


公開URLの後に記載されているURLにブラウザでアクセスすると、streamlitのUIが表示されます。

app.pyのコメントアウトされている箇所を編集することで、UIがどの様に変化するか確認してみましょう。

streamlitの公式ページには、ギャラリーページがあります。

streamlitを使うとpythonという一つの言語であっても、様々なUIを実現できることがわかると思います。

https://streamlit.io/gallery

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [9]:
from pyngrok import ngrok
ngrok.kill()

# 02_streamlit_app


ディレクトリ「02_streamlit_app」に移動します。

In [16]:
%cd /home/tmiya/src/github.com/silbull/lecture-ai-engineering/day1/02_streamlit_app

/home/tmiya/src/github.com/silbull/lecture-ai-engineering/day1/02_streamlit_app


必要なライブラリをインストールします。

In [17]:
%%capture
!pip install -r requirements.txt

ngrokとhuggigfaceのトークンを使用して、認証を行います。

In [ ]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

stramlitでHuggingfaceのトークン情報を扱うために、streamlit用の設定ファイル（.streamlit）を作成し、トークンの情報を格納します。

In [ ]:
# .streamlit/secrets.toml ファイルを作成
import os
import toml

# 設定ファイルのディレクトリ確保
os.makedirs('.streamlit', exist_ok=True)

# 環境変数から取得したトークンを設定ファイルに書き込む
secrets = {
    "huggingface": {
        "token": os.environ.get("HUGGINGFACE_TOKEN", "")
    }
}

# 設定ファイルを書き込む
with open('.streamlit/secrets.toml', 'w') as f:
    toml.dump(secrets, f)

アプリを起動します。

02_streamlit_appでは、Huggingfaceからモデルをダウンロードするため、初回起動には2分程度時間がかかります。

この待ち時間を利用して、app.pyのコードを確認してみましょう。

In [ ]:
import subprocess
import time
from pyngrok import ngrok

# streamlit起動（8501番ポート）
proc = subprocess.Popen(["streamlit", "run", "app.py"])

# 起動をちょっと待つ（本来はポート確認がベター）
time.sleep(5)

# ngrokトンネル作成
public_url = ngrok.connect(8501)
print(f"🔗 公開URL: {public_url}")

try:
    proc.wait()
except KeyboardInterrupt:
    print("🛑 終了中...")
    proc.terminate()
    ngrok.kill()
    print("🔒 ngrokトンネルを終了しました。")

アプリケーションの機能としては、チャット機能や履歴閲覧があります。

これらの機能を実現するためには、StreamlitによるUI部分だけではなく、SQLiteを使用したチャット履歴の保存やLLMのモデルを呼び出した推論などの処理を組み合わせることで実現しています。

- **`app.py`**: アプリケーションのエントリーポイント。チャット機能、履歴閲覧、サンプルデータ管理のUIを提供します。
- **`ui.py`**: チャットページや履歴閲覧ページなど、アプリケーションのUIロジックを管理します。
- **`llm.py`**: LLMモデルのロードとテキスト生成を行うモジュール。
- **`database.py`**: SQLiteデータベースを使用してチャット履歴やフィードバックを保存・管理します。
- **`metrics.py`**: BLEUスコアやコサイン類似度など、回答の評価指標を計算するモジュール。
- **`data.py`**: サンプルデータの作成やデータベースの初期化を行うモジュール。
- **`config.py`**: アプリケーションの設定（モデル名やデータベースファイル名）を管理します。
- **`requirements.txt`**: このアプリケーションを実行するために必要なPythonパッケージ。

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [ ]:
from pyngrok import ngrok
ngrok.kill()

# 03_FastAPI

ディレクトリ「03_FastAPI」に移動します。

In [ ]:
%cd /content/lecture-ai-engineering/day1/03_FastAPI

必要なライブラリをインストールします。

In [ ]:
%%capture
!pip install -r requirements.txt

ngrokとhuggigfaceのトークンを使用して、認証を行います。

In [ ]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

アプリを起動します。

「02_streamlit_app」から続けて「03_FastAPI」を実行している場合は、モデルのダウンロードが済んでいるため、すぐにサービスが立ち上がります。

「03_FastAPI」のみを実行している場合は、初回の起動時にモデルのダウンロードが始まるので、モデルのダウンロードが終わるまで数分間待ちましょう。

In [ ]:
!python app.py

FastAPIが起動すると、APIとクライアントが通信するためのURL（エンドポイント）が作られます。

URLが作られるのと合わせて、Swagger UIというWebインターフェースが作られます。

Swagger UIにアクセスすることで、APIの仕様を確認できたり、APIをテストすることができます。

Swagger UIを利用することで、APIを通してLLMを動かしてみましょう。

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [ ]:
from pyngrok import ngrok
ngrok.kill()